In [0]:
!pip install category_encoders

     |████████████████████████████████| 102kB 1.2MB/s 


In [0]:
import pandas as pd   #import pandas
import numpy as np    #import numpy
from sklearn.preprocessing import LabelEncoder #import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

In [0]:
df = pd.read_csv('/content/drive/My Drive/hotel_bookings.csv')  #read the data

In [0]:
df.head()  #check the head

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [0]:
#check the shape of the dataset
print('Data has {} rows of and {} of columns '.format(df.shape[0], df.shape[1]))

Data has 119390 rows of and 32 of columns 


In [0]:
#check info of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

Here we have 32 columns. 
* is_canceled is our target feature.
* There are 11 categorical features.
* There are 21 numeric features

In [0]:
#store categorical variables in c_col
c_col = df[['is_canceled','hotel','arrival_date_month','meal','country','market_segment','distribution_channel',
         'reserved_room_type'	,'assigned_room_type','deposit_type','customer_type','reservation_status']]

Check categorical variable one by one, classify them as nominal or ordinal and apply the proper encoding technique.

In [0]:
#check values_counts in hotel
c_col['hotel'].value_counts()

City Hotel      79330
Resort Hotel    40060
Name: hotel, dtype: int64

* There are 2 labels.
* Thers is no ordering (Nominal categorical variable)
* We will aapply LabelEncoding here.

In [0]:
le = LabelEncoder()   #create the instance of LabelEncoder
c_col['hotel_en'] = le.fit_transform(c_col['hotel'])   # apply label encoding 
c_col.drop('hotel', axis=1, inplace=True) #drop the hotel column
c_col['hotel_en'].value_counts()   

0    79330
1    40060
Name: hotel_en, dtype: int64

Now check 'arrival_date_month'

In [0]:
#check the value counts in arrival_date_month
c_col['arrival_date_month'].value_counts()

August       13877
July         12661
May          11791
October      11160
April        11089
June         10939
September    10508
March         9794
February      8068
November      6794
December      6780
January       5929
Name: arrival_date_month, dtype: int64

The labels here are months of the years.

We will use map to encode the month labels.
* January -------> 0
* February ------> 1
* March  --------> 2
* April  --------> 3
* May   ---------> 4
* June  ---------> 5
* July  ---------> 6
* August --------> 7
* September -----> 8 
* October  ------> 9
* November ------> 10
* December ------> 11


In [0]:
#prepare a dict for mapping
month_dict = {'January': 0, 'February' : 1,'March' : 2,'April' : 3, 'May' : 4, 'June' : 5, 
              'July': 6, 'August' : 7,'September' : 10, 'October': 9, 'November' : 10, 'December' : 11}
#map month_dict on arrival_date_month
c_col['arrival_date_month_en'] = c_col['arrival_date_month'].map(month_dict)

c_col.drop('arrival_date_month',axis = 1, inplace = True)

c_col['arrival_date_month_en'].value_counts()

10    17302
7     13877
6     12661
4     11791
9     11160
3     11089
5     10939
2      9794
1      8068
11     6780
0      5929
Name: arrival_date_month_en, dtype: int64

In [0]:
c_col['meal'].value_counts()

BB           92310
HB           14463
SC           10650
Undefined     1169
FB             798
Name: meal, dtype: int64

* BB -- Bed & Breakfast
* HB -- Breakfast & Dinner
* SC -- No Meal Package
* FB -- Breakfast,lunch and dinner
* Undefined -- No Meal Package

Here the labels are nominal we will apply OneHotEncoding here.

In [0]:
meal_en = pd.get_dummies(c_col['meal'],drop_first=True) #apply pd.get_dummies
meal_en.head(10)

,FB,HB,SC,Undefined
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,1,0,0,0
8,0,0,0,0
9,0,1,0,0


In [0]:
#check the country column
c_col['country'].nunique()

177

Here we have 177 unique labels. Labels are the country codes here (nominal).

If we apply one hot encoding on 177 features we will have 176 new features.
Here we will use frequency encoding. 

In [0]:
fe = c_col['country'].value_counts(ascending=True)/len(c_col)  #count the relative frequency
c_col['country_en'] = c_col['country'].map(fe) #map with fe
c_col.drop('country',axis=1,inplace=True)
c_col['country_en'].head(10)

0    0.406986
1    0.406986
2    0.101591
3    0.101591
4    0.101591
5    0.101591
6    0.406986
7    0.406986
8    0.406986
9    0.406986
Name: country_en, dtype: float64

Check the market segment

In [0]:
#check the value_counts
c_col['market_segment'].value_counts()

Online TA        56477
Offline TA/TO    24219
Groups           19811
Direct           12606
Corporate         5295
Complementary      743
Aviation           237
Undefined            2
Name: market_segment, dtype: int64

In [0]:
c_col['reserved_room_type'].value_counts()

A    85994
D    19201
E     6535
F     2897
G     2094
B     1118
C      932
H      601
P       12
L        6
Name: reserved_room_type, dtype: int64

In [0]:
c_col['assigned_room_type'].value_counts()

A    74053
D    25322
E     7806
F     3751
G     2553
C     2375
B     2163
H      712
I      363
K      279
P       12
L        1
Name: assigned_room_type, dtype: int64

* we will apply Binary encoding on 3 columns as they are nominal categorical variable ('market_segment', 'assigned_room_type','reserved_room_type').

In [0]:
import category_encoders as ce #create instance of category_encoders
encoder = ce.BinaryEncoder(cols= ['market_segment','reserved_room_type','assigned_room_type'])        #create instance of binary enocder
market_binary = encoder.fit_transform(c_col)
market_binary.head(5) 

,is_canceled,meal,market_segment_0,market_segment_1,market_segment_2,market_segment_3,distribution_channel,reserved_room_type_0,reserved_room_type_1,reserved_room_type_2,reserved_room_type_3,reserved_room_type_4,assigned_room_type_0,assigned_room_type_1,assigned_room_type_2,assigned_room_type_3,assigned_room_type_4,deposit_type,customer_type,reservation_status,hotel_en,arrival_date_month_en,country_en
0,0,BB,0,0,0,1,Direct,0,0,0,0,1,0,0,0,0,1,No Deposit,Transient,Check-Out,1,6,0.406986
1,0,BB,0,0,0,1,Direct,0,0,0,0,1,0,0,0,0,1,No Deposit,Transient,Check-Out,1,6,0.406986
2,0,BB,0,0,0,1,Direct,0,0,0,1,0,0,0,0,0,1,No Deposit,Transient,Check-Out,1,6,0.101591
3,0,BB,0,0,1,0,Corporate,0,0,0,1,0,0,0,0,1,0,No Deposit,Transient,Check-Out,1,6,0.101591
4,0,BB,0,0,1,1,TA/TO,0,0,0,1,0,0,0,0,1,0,No Deposit,Transient,Check-Out,1,6,0.101591


we will apply pd.get_dummies on the remaining 4 categorical features  as they are  ordinal and has only 2-3 labels.

In [0]:
#apply get_dummies
cols = ['reservation_status','customer_type','deposit_type','distribution_channel']
cols_encode = pd.get_dummies(c_col, columns = cols,drop_first=True)


That's it for now. If you liked this compilation of different encoding techniques with applications on real world datasets do let us now.

In [0]:
                                                  # I can be contacted at maan46.ms@gmail.com
                                                  # LinkedIn - https://www.linkedin.com/in/manvendra-singh-228b85147/